# All installs

# All imports and setup

In [108]:
from pyspark.sql import SparkSession , functions as F

spark = SparkSession.builder.appName('Salaries')\
    .config('spark.driver.extraClassPath', '/usr/lib/jvm/java-17-openjdk-amd64/lib/postgresql-42.5.0.jar')\
    .getOrCreate()


In [109]:
Salaries = spark.read.json('Data/Salaries.json')
Salaries.show(5)

+-------------+---------+--------+-----------------+---+--------------------+-----+---------+-----------+------+---------+----------------+----+---------------+
|       Agency|  BasePay|Benefits|     EmployeeName| Id|            JobTitle|Notes| OtherPay|OvertimePay|Status| TotalPay|TotalPayBenefits|Year|_corrupt_record|
+-------------+---------+--------+-----------------+---+--------------------+-----+---------+-----------+------+---------+----------------+----+---------------+
|San Francisco|167411.18|        |   NATHANIEL FORD|  1|GENERAL MANAGER-M...|     |400184.25|        0.0|      |567595.43|       567595.43|2011|           null|
|San Francisco|155966.02|        |     GARY JIMENEZ|  2|CAPTAIN III (POLI...|     |137811.38|  245131.88|      |538909.28|       538909.28|2011|           null|
|San Francisco|212739.13|        |   ALBERT PARDINI|  3|CAPTAIN III (POLI...|     |  16452.6|  106088.18|      |335279.91|       335279.91|2011|           null|
|San Francisco|  77916.0|        |

In [110]:
Salaries.printSchema()

root
 |-- Agency: string (nullable = true)
 |-- BasePay: string (nullable = true)
 |-- Benefits: string (nullable = true)
 |-- EmployeeName: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- OtherPay: string (nullable = true)
 |-- OvertimePay: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- TotalPay: string (nullable = true)
 |-- TotalPayBenefits: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)



# 1. Find the average salaries of each job position(assuming salary is the total of all pays).

In [111]:

Avg_sal_job=Salaries.groupBy('jobTitle').agg(F.round(F.avg('Totalpay'),2).alias('avg_salary')).sort('jobTitle')

Avg_sal_job=Avg_sal_job.filter(Avg_sal_job.avg_salary.isNotNull()) # removes null values

Avg_sal_job.show(5)

print ('count: There are {} job titles'.format(Avg_sal_job.count()))

################ CSV ################
Avg_sal_job.coalesce(1).write.csv('Output/Salaries/Avg_sal_job',header=True,mode='overwrite')


############### Postgres ###############
Avg_sal_job.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Salaries',driver='org.postgresql.Driver',dbtable='Avg_sal_job',user='amrit',password='1234').mode('overwrite').save()


+--------------------+----------+
|            jobTitle|avg_salary|
+--------------------+----------+
|       ACCOUNT CLERK|  44035.66|
|          ACCOUNTANT|  47429.27|
|   ACCOUNTANT INTERN|  29031.74|
|ACPO,JuvP, Juv Pr...|  62290.78|
|       ACUPUNCTURIST|   67594.4|
+--------------------+----------+
only showing top 5 rows

count: There are 2158 job titles


# 2. Which job title has the highest full time employees?



In [112]:
Title_FT_Count=Salaries.filter(Salaries.Status=='FT').groupBy('jobTitle').agg(F.count('jobTitle').alias('count')).sort('count',ascending=False)

Title_FT_Count.show(5)

print('Highest full time employee and count:',format(Title_FT_Count.first()))


############### CSV ################
Title_FT_Count.coalesce(1).write.csv('Output/Salaries/Title_FT_Count',header=True,mode='overwrite')

############### Postgres ###############
Title_FT_Count.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Salaries',driver='org.postgresql.Driver',dbtable='Title_FT_Count',user='amrit',password='1234').mode('overwrite').save()


+----------------+-----+
|        jobTitle|count|
+----------------+-----+
|Transit Operator| 1524|
|     Firefighter|  738|
|Police Officer 3|  642|
|       Custodian|  565|
|  Deputy Sheriff|  552|
+----------------+-----+
only showing top 5 rows

Highest full time employee and count: Row(jobTitle='Transit Operator', count=1524)


# 3. List the name of employees who work for the police department?


In [113]:
Emp_Police=Salaries.filter(Salaries.JobTitle.contains('POLICE')).select('EmployeeName').sort('EmployeeName')

Emp_Police.show(5)

print('count: There are {} employees in police department'.format(Emp_Police.count())) 

############### CSV ################
Emp_Police.coalesce(1).write.csv('Output/Salaries/Emp_Police',header=True,mode='overwrite')

############### Postgres ###############
Emp_Police.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Salaries',driver='org.postgresql.Driver',dbtable='Emp_Police',user='amrit',password='1234').mode('overwrite').save()


+---------------+
|   EmployeeName|
+---------------+
|AARON BALLONADO|
|   AARON COWHIG|
|  AARON FISCHER|
|    AARON FOLTZ|
|   AARON LOZADA|
+---------------+
only showing top 5 rows

count: There are 2512 employees in police department


# 4. Find the job titles along with the employees name and ids.

In [114]:
Title_Name_Id=Salaries.select('JobTitle','EmployeeName','Id').sort('Employeename')

Title_Name_Id=Title_Name_Id.filter(Title_Name_Id.EmployeeName.isNotNull())

Title_Name_Id=Title_Name_Id.withColumn('EmployeeName',F.regexp_replace('EmployeeName','\s+',' ')) # removes extra spaces in EmployeeName

Title_Name_Id_lower=Title_Name_Id.withColumn('JobTitle',F.lower(F.col('JobTitle'))).withColumn('EmployeeName',F.lower(F.col('EmployeeName'))) 

Title_Name_Ids=Title_Name_Id_lower.groupBy('EmployeeName','JobTitle').agg(F.concat_ws(',',F.collect_list('Id')).alias('Id(s)')).sort('EmployeeName')

Title_Name_Ids.show(5)


########## CSV ##########
Title_Name_Ids.coalesce(1).write.csv('Output/Salaries/Title_Name_Ids',header=True,mode='overwrite')

########## Postgres ##########
Title_Name_Ids.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Salaries',driver='org.postgresql.Driver',dbtable='Title_Name_Ids',user='amrit',password='1234').mode('overwrite').save()



+--------------------+--------------------+--------------------+
|        EmployeeName|            JobTitle|               Id(s)|
+--------------------+--------------------+--------------------+
|    a bernard fatooh|sheriff's propert...|29354,102844,6637...|
|a elizabeth march...|     principal clerk|               28470|
|       a jamil niazi|is engineer - pri...|               12419|
|         a k finizio|customer service ...|              140928|
|a. james robertso...|superior court judge|  29449,103641,66488|
+--------------------+--------------------+--------------------+
only showing top 5 rows



# 5. Find the number of employees in each job title. 

In [115]:
Title_Count=Salaries.groupBy('JobTitle').agg(F.count('JobTitle').alias('count')).sort('count',ascending=False)

Title_Count.show(5)

print('Highest job title and count:',format(Title_Count.first()))

########## CSV ##########
Title_Count.coalesce(1).write.csv('Output/Salaries/Title_Count',header=True,mode='overwrite')

########## Postgres ##########
Title_Count.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Salaries',driver='org.postgresql.Driver',dbtable='Title_Count',user='amrit',password='1234').mode('overwrite').save()

+--------------------+-----+
|            JobTitle|count|
+--------------------+-----+
|    Transit Operator| 7036|
|       Special Nurse| 4389|
|    Registered Nurse| 3736|
|Public Svc Aide-P...| 2518|
|    Police Officer 3| 2421|
+--------------------+-----+
only showing top 5 rows

Highest job title and count: Row(JobTitle='Transit Operator', count=7036)


# 6. List out the names and positions of employees whose total pay is greater than 180000.

In [116]:
Name_Pos_180k=Salaries.filter(Salaries.TotalPay>180000).select('EmployeeName','JobTitle','TotalPay').sort('TotalPay')

Name_Pos_180k.show(5)

########### CSV ###########
Name_Pos_180k.coalesce(1).write.csv('Output/Salaries/Name_Pos_180k',header=True,mode='overwrite')

########### Postgres ###########
Name_Pos_180k.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Salaries',driver='org.postgresql.Driver',dbtable='Name_Pos_180k',user='amrit',password='1234').mode('overwrite').save()



+----------------+--------------------+---------+
|    EmployeeName|            JobTitle| TotalPay|
+----------------+--------------------+---------+
| Shirley C Banks|  Nurse Practitioner|180003.53|
|Eileen McCrystle|     Insp, Fire Dept|180004.84|
|     PEI JIAN LI|ELECTRICAL TRANSI...|180016.42|
|   Spencer Nakao|         Firefighter|180023.98|
|Grace  Fortaleza|    Registered Nurse|180026.97|
+----------------+--------------------+---------+
only showing top 5 rows



# 7. List the names and ids of employees who have never done overtime.


In [117]:
Name_Id_No_OT=Salaries.filter(Salaries.OvertimePay=='0.00').select('EmployeeName','Id').sort('EmployeeName')

Name_Id_No_OT.show(5)

print('count: There are {} employees who have never done overtime'.format(Name_Id_No_OT.count()))

########### CSV ###########
Name_Id_No_OT.coalesce(1).write.csv('Output/Salaries/Name_Id_No_OT',header=True,mode='overwrite')

########### Postgres ###########
Name_Id_No_OT.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Salaries',driver='org.postgresql.Driver',dbtable='Name_Id_No_OT',user='amrit',password='1234').mode('overwrite').save()


+--------------------+------+
|        EmployeeName|    Id|
+--------------------+------+
|    A Bernard Fatooh|140526|
|         A K Finizio|140928|
|Aaliyah Javae'Mer...|146943|
|     Aaliyah V Hogue|141683|
|    Aaron A Hipolito|145251|
+--------------------+------+
only showing top 5 rows

count: There are 19448 employees who have never done overtime
